In [1]:
import bashlex
import copy
import re
import os

from tqdm import tqdm

In [2]:
re_command_filter = "C"
re_error_filter = "X"
re_start_filter = "S" 

In [3]:
scientists_dir = "./unix-data/computer-scientists/"
experienced_dir = "./unix-data/experienced-programmers/"
non_programmers_dir = "./unix-data/non-programmers/"
novice_dir = "./unix-data/novice-programmers/"

scientists_files = os.listdir(scientists_dir)
experienced_files = os.listdir(experienced_dir)
non_programmers_files = os.listdir(non_programmers_dir)
novice_files = os.listdir(novice_dir)

scientist_commands = []
experienced_commands = []
non_programmers_commands = []
novice_commands = []

scientist_parsed = []
experienced_parsed = []
non_programmers_parsed = []
novice_parsed = []

Generates list of files

In [4]:
for i in range(len(scientists_files)):
    scientists_files[i] = scientists_dir + scientists_files[i]

for i in range(len(experienced_files)):
    experienced_files[i] = experienced_dir + experienced_files[i]

for i in range(len(non_programmers_files)):
    non_programmers_files[i] = non_programmers_dir + non_programmers_files[i]

for i in range(len(novice_files)):
    novice_files[i] = novice_dir + novice_files[i]

Parse function that returns nested list of parsed commands based on session

In [16]:
def parse_commands_per_session(command_list, parsed_list, files_list):
    for file_path in tqdm(files_list):
        file1 = open(file_path, encoding="ISO-8859-1")
        lines = file1.readlines()

        command_sublist = []
        parsed_sublist = []

        for line in range(len(lines)):
            
            if re.match(re_start_filter, lines[line]) is not None:
                if len(command_sublist) != 0 and len(parsed_sublist) != 0:
                    command_list.append(copy.deepcopy(command_sublist))
                    parsed_list.append(copy.deepcopy(parsed_sublist))


                command_sublist = []
                parsed_sublist = []
            
            if re.match(re_command_filter, lines[line]) is not None:
                command_sublist.append(lines[line][2:-1])
                
                try:
                    parts = list(bashlex.split(command_sublist[-1]))
                    parsed_sublist.append(parts)
                except Exception as inst:
                    command_sublist.pop(-1)
                    
    
    # Not necessary as list is created by ref parameter 
    return command_list
            

Function that parses just by command

In [17]:
def parse_commands(command_list, parsed_list, files_list):
    for file_path in tqdm(files_list): 
        file1 = open(file_path, encoding="ISO-8859-1")
        lines = file1.readlines()

        for line in range(len(lines)):
            if re.match(re_command_filter, lines[line]) is not None:
                command_list.append(lines[line][2:-1])
                
                try:
                    parts = bashlex.parse(command_list[-1])
                    parsed_list.append(parts)
                except Exception as inst:
                    command_list.pop(-1)

    return command_list

Function that parses commands and groups into subsets of `subset_size` meaning command chains are `subset_size` commands long.

In [18]:
def parse_commands_into_subsets(command_list, parsed_list, files_list, subset_size):
    command_list = copy.deepcopy(parse_commands(command_list, parsed_list, files_list))

    filter_escaped = lambda x: (x and x.isprintable())
    command_list = list(filter(filter_escaped, command_list))

    result = []

    for i in range(0, len(command_list), subset_size):
        result.append(command_list[i:i+subset_size])

    command_list = result

    return command_list


In [19]:
scientist_commands = []
experienced_commands = []
non_programmers_commands = []
novice_commands = []

scientist_commands = parse_commands_per_session(scientist_commands, scientist_parsed, scientists_files)
experienced_commands = parse_commands_per_session(experienced_commands, experienced_parsed, experienced_files)
non_programmers_commands = parse_commands_per_session(non_programmers_commands, non_programmers_parsed, non_programmers_files)
novice_commands = parse_commands_per_session(novice_commands, novice_parsed, novice_files)

filter_empty = lambda x: (len(x) > 0)

scientist_commands = list(filter(filter_empty, scientist_commands))
experienced_commands = list(filter(filter_empty, experienced_commands))
non_programmers_commands = list(filter(filter_empty, non_programmers_commands))
novice_commands = list(filter(filter_empty, novice_commands))

100%|██████████| 56/56 [00:06<00:00,  8.91it/s]


In [20]:
subset_size = 5

scientist_commands = []
experienced_commands = []
non_programmers_commands = []
novice_commands = []

scientist_commands_subsets = parse_commands_into_subsets(scientist_commands, scientist_parsed, scientists_files, subset_size)
experienced_commands_subsets = parse_commands_into_subsets(experienced_commands, experienced_parsed, experienced_files, subset_size)
non_programmers_commands_subsets = parse_commands_into_subsets(non_programmers_commands, non_programmers_parsed, non_programmers_files, subset_size)
novice_commands_subsets = parse_commands_into_subsets(novice_commands, novice_parsed, novice_files, subset_size)

filter_empty = lambda x: (x is not None)

scientist_commands_subsets = list(filter(filter_empty, scientist_commands_subsets))
experienced_commands_subsets = list(filter(filter_empty, experienced_commands_subsets))
non_programmers_commands_subsets = list(filter(filter_empty, non_programmers_commands_subsets))
novice_commands_subsets = list(filter(filter_empty, novice_commands_subsets))

100%|██████████| 56/56 [00:11<00:00,  5.04it/s]


In [21]:
print(len(scientist_commands_subsets))
print(len(experienced_commands_subsets))
print(len(non_programmers_commands_subsets))
print(len(novice_commands_subsets))

print(scientist_commands_subsets[1])

24925
14790
5076
15336
['ls', 'm ken', 'gc', 'ls -l', 'alias']


Defunct, predicts next argument in command with this structure

In [16]:
from graph import Node

def construct_graph_defunct(parsed_commands, command_dict = {}):

    for cmd in parsed_commands:

        cur_node = None

        if command_dict.get(cmd[0]) is None:
            cur_node = Node(command = cmd[0], frequency = 1)
            command_dict[cmd[0]] = cur_node
        else:
            command_dict.get(cmd[0]).frequency += 1
            cur_node = command_dict.get(cmd[0])

        for i in range(1, len(cmd)):
            child_node = None

            if cur_node.children.get(cmd[i]) is None:
                child_node = Node(cmd[i], 1)
                cur_node.children[cmd[i]] = child_node
            else:
                cur_node.children.get(cmd[i]).frequency += 1
                child_node = cur_node.children.get(cmd[i])
            
            cur_node = child_node
        
    return command_dict


Graph structure for next command. Use command list instead of bashlex output since we are comparing whole commands as nodes in a graph.

Making command dict key the program (e.g. `cat`) and value the node with that program and have that nodes children be the full command. And then have the children of those nodes be determined by fuzzy matching. So a node can be a child to many parents if its fuzzy matched.

In [4]:
from graph import Node

def construct_graph(command_list, command_dict={}):

    filter_empty = lambda x: (len(x) > 0)
    cur_node = None
    child_node = None
    
    for session in command_list:
        # remove commands of length 0 ( not good to modify list within loop )
        
        session = list(filter(filter_empty, session))

        first_cmd = session[0]

        program = session[0].split()[0]

        if command_dict.get(program) is None:
            cur_node = Node(program=program, frequency=1)
            command_dict[program] = cur_node 
        else:
            cur_node = command_dict.get(program)
            cur_node.frequency += 1

        if cur_node.commands.get(first_cmd) is None:
            cur_node.commands[first_cmd] = 1
        else:
            cur_node.commands[first_cmd] += 1

        for cmd in range(1, len(session)):

            if session[cmd].isprintable() is False:
                break
            else:
                program = session[cmd].split()[0]

            if cur_node.children.get(program) is None:
                child_node = Node(program = program, frequency=1)
                cur_node.children[program] = child_node
            else:
                child_node = cur_node.children.get(program)
                child_node.frequency += 1
            
            if child_node.commands.get(session[cmd]) is None:
                child_node.commands[session[cmd]] = 1
            else:
                child_node.commands[session[cmd]] += 1
            
            cur_node = child_node
        
                
        
    return command_dict


Takes list of commands and observes last 4 to predict 5th

In [5]:
def get_prediction(command_list, graph):
    if len(command_list) == 0:
        return None

    commands = command_list[-3:-1]
    previous_command = command_list[-1]

    if len(commands) == 0:
        return None

    program = commands[0].split()[0]
    
    if graph.get(program) is None:
        return None
    else:
        node = graph[program]

    for command in commands[1:]:
        program = command.split()[0]
        if node.children.get(program) is not None:
            node = node.children[program]
        else:
            return None

    return node.get_prediction(previous_command)


In [27]:
from thefuzz import fuzz
from thefuzz import process

import pprint
pp = pprint.PrettyPrinter(indent=4)

def test_train_split(command_list, split_ratio):
    test_data = command_list[int((len(command_list) + 1) * split_ratio):]
    train_data = command_list[:int((len(command_list) + 1) * split_ratio)]

    return test_data, train_data

def get_accuracy(command_subsets):
    split_ratio = .8
    test_data, train_data = test_train_split(command_subsets, split_ratio)

    train_graph = construct_graph(train_data)

    test_size = len(test_data)
    correct = 0

    for commands in test_data:
        results = get_prediction(commands, train_graph)
        """
        if fuzz.ratio(results, commands[-1]) > 75:
            correct += 1
        """
        
        if results is not None:
            for result in results:
                if fuzz.partial_ratio(result[0], commands[-1]) > 75:
                    correct += 1
                    break
                else:
                    print("result", result[0], "|", "command sequence:", commands)

    return '{:.2f}%'.format(100 * correct/test_size)

# print out when not matching
# be able to look up graph -> for pipes -> and trace through not matching
# ignore alias commands but process aliases
# create example of things working and not working

# for pipes want to know fraction of time did not have prediction
# find and example null predictions

# exhaustive algorithm for all combinations 
# graph statistics with distributions of counts with command chains

# colearning, predicting one person's commands, using one person to predict another person


Cyverse command pull and parse

In [ ]:
import requests
import time

url = "https://us-south.functions.appdomain.cloud/api/v1/web/ORG-UNC-dist-seed-james_dev/cyverse/get-cyverse-log"
query = {'body': {'log_type':'Bash', 'password': 'password', 'skip': '0', 'limit':'100'}}

response_dict = {}

def get_cyverse_commands():
    for i in range(0, 1000, 100):
        query = {'body': {'log_type':'Bash', 'password': 'password', 'skip': str(i), 'limit':'100'}}
        response = requests.get(url, headers = {"Content-Type": "application/json"}, json = query)
        print(i, response.status_code)
        response_dict.update(response.json())
        time.sleep(50)

    return response_dict

get_cyverse_commands()


In [ ]:
cyverse_commands = []

for log in response_dict['logs']:
    for command in log['log']['commands']:
        if 'full_command' in command:
            cyverse_commands.append(command['full_command'])

len(cyverse_commands)

In [ ]:
def parse_cyverse_subsets(command_list, subset_size):
    result = []

    for i in range(0, len(command_list), subset_size):
        result.append(command_list[i:i+subset_size])

    return result

cyverse_commands_subsets = parse_cyverse_subsets(cyverse_commands, 5)

get_accuracy(cyverse_commands_subsets)

Replace args result

In [25]:
print("Scientists accuracy:", get_accuracy(scientist_commands_subsets))
print("Experienced accuracy:", get_accuracy(experienced_commands_subsets))
print("Non programmers accuracy:", get_accuracy(non_programmers_commands_subsets))
print("Novice programmers accuracy:", get_accuracy(novice_commands_subsets))

#all_data = scientist_commands_subsets + experienced_commands_subsets + non_programmers_commands_subsets + novice_commands_subsets

#print("All data accuracy:", get_accuracy(all_data))

#print("Cyverse accuracy", get_accuracy(cyverse_commands_subsets))

Scientists accuracy: 20.40%
Experienced accuracy: 18.53%
Non programmers accuracy: 33.60%
Novice programmers accuracy: 27.62%


In [7]:
from parse import Parser

parser = Parser()


100%|██████████| 56/56 [00:05<00:00, 10.32it/s]


In [8]:
subset_size = 5

parser_scientist_commands_subsets = parser.replace_args(parser.scientists_commands)
parser_scientist_commands_subsets = parser.parse_commands_into_subsets(parser_scientist_commands_subsets, subset_size)

parser_experienced_commands_subsets = parser.replace_args(parser.experienced_commands)
parser_experienced_commands_subsets = parser.parse_commands_into_subsets(parser_experienced_commands_subsets, subset_size)

parser_non_commands_subsets = parser.replace_args(parser.non_programmers_commands)
parser_non_commands_subsets = parser.parse_commands_into_subsets(parser_non_commands_subsets, subset_size)

parser_novice_commands_subsets = parser.replace_args(parser.novice_commands)
parser_novice_commands_subsets = parser.parse_commands_into_subsets(parser_novice_commands_subsets, subset_size)


print("Scientists accuracy:", get_accuracy(parser_scientist_commands_subsets))
print("Experienced accuracy:", get_accuracy(parser_experienced_commands_subsets))
print("Non programmers accuracy:", get_accuracy(parser_non_commands_subsets))
print("Novice programmers accuracy:", get_accuracy(parser_novice_commands_subsets))

Scientists accuracy: 37.20%
Experienced accuracy: 29.84%
Non programmers accuracy: 66.54%
Novice programmers accuracy: 59.46%


In [14]:
science = parser.filter_commands_with_pipe(parser.scientists_commands)
experienced = parser.filter_commands_with_pipe(parser.experienced_commands)
non = parser.filter_commands_with_pipe(parser.non_programmers_commands)
novice = parser.filter_commands_with_pipe(parser.novice_commands)

science_pipes = parser.expand_piped_commands(science)
experienced_pipes = parser.expand_piped_commands(experienced)
non_pipes = parser.expand_piped_commands(non)
novice_pipes = parser.expand_piped_commands(novice)

print(get_accuracy(science_pipes))
print(get_accuracy(experienced_pipes))
print(get_accuracy(non_pipes))
print(get_accuracy(novice_pipes))

64.80%
85.97%
97.29%
55.56%


In [9]:
subset_size = 5

science_replaced = parser.replace_args(parser.scientists_commands)
science_replaced = parser.parse_commands_into_subsets(science_replaced, subset_size)

experienced_replaced = parser.replace_args(parser.experienced_commands)
experienced_replaced = parser.parse_commands_into_subsets(experienced_replaced, subset_size)

non_replaced = parser.replace_args(parser.non_programmers_commands)
non_replaced = parser.parse_commands_into_subsets(non_replaced, subset_size)

novice_replaced = parser.replace_args(parser.novice_commands)
novice_replaced = parser.parse_commands_into_subsets(novice_replaced, subset_size)

print("Scientist replaced accuracy:", get_accuracy(science_replaced))
print("Experienced replaced accuracy:", get_accuracy(experienced_replaced))
print("Non programmer replaced accuracy:", get_accuracy(non_replaced))
print("Novice replaced accuracy:", get_accuracy(novice_replaced))

Scientist replaced accuracy: 41.21%
Experienced replaced accuracy: 30.11%
Non programmer replaced accuracy: 66.04%
Novice replaced accuracy: 59.46%


In [28]:
print("Scientist replaced accuracy:", get_accuracy(science_replaced))
print("Experienced replaced accuracy:", get_accuracy(experienced_replaced))
print("Non programmer replaced accuracy:", get_accuracy(non_replaced))
print("Novice replaced accuracy:", get_accuracy(novice_replaced))

result cd | command sequence: ['ls', 'e $0', 'roff -a2 $0 $1', 'bg', 'fg']
result ps -x | command sequence: ['ls', 'e $0', 'roff -a2 $0 $1', 'bg', 'fg']
result e $0 | command sequence: ['ls', 'e $0', 'roff -a2 $0 $1', 'bg', 'fg']
result rm $0 | command sequence: ['cd $0', 'ps', 'rm $0', 'e $0', 'cd $0']
result m $0 | command sequence: ['alias $0 $1 $2 $3 $4 $5 $6 $7 $8 -F', 'alias $0 $1 $2 $3 $4 $5 $6 $7 $8 $9 -F"', 'dir', 'ls -F', 'more $0']
result del $0 | command sequence: ['alias $0 $1 $2 $3 $4 $5 $6 $7 $8 -F', 'alias $0 $1 $2 $3 $4 $5 $6 $7 $8 $9 -F"', 'dir', 'ls -F', 'more $0']
result ls $0 | command sequence: ['alias $0 $1 $2 $3 $4 $5 $6 $7 $8 -F', 'alias $0 $1 $2 $3 $4 $5 $6 $7 $8 $9 -F"', 'dir', 'ls -F', 'more $0']
result dir | command sequence: ['alias $0 $1 $2 $3 $4 $5 $6 $7 $8 -F', 'alias $0 $1 $2 $3 $4 $5 $6 $7 $8 $9 -F"', 'dir', 'ls -F', 'more $0']
result limits | command sequence: ['alias $0 $1 $2 $3 $4 $5 $6 $7 $8 -F', 'alias $0 $1 $2 $3 $4 $5 $6 $7 $8 $9 -F"', 'dir', '

In [31]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

science_replaced = parser.replace_args(parser.scientists_commands) 

In [15]:
from parse import Parser
import pprint
pp = pprint.PrettyPrinter(indent=4)

parser = Parser()

science2 = parser.filter_commands_with_pipe(parser.scientists_commands)
experienced2 = parser.filter_commands_with_pipe(parser.experienced_commands)
non2 = parser.filter_commands_with_pipe(parser.non_programmers_commands)
novice2 = parser.filter_commands_with_pipe(parser.novice_commands)

def replace_arg_pipes(parsed_list):

    parsed_list_replaced = []
    arg_replacements = ["$" + str(i) for i in range(100)]

    for command in range(len(parsed_list)):
        command_split = parsed_list[command].split()

        arg_counter = 0

        for i in range(1, len(command_split)):
            if command_split[i] == "|":
                pass

            elif not command_split[i].startswith("-") and not command_split[i-1] == '|' and not command_split[i] == '&':
                #replace only file name, not file extension
                command_split_by_extension = command_split[i].split(".")
                command_split_by_extension[0] = arg_replacements[arg_counter]
                command_split[i] = ''.join(command_split_by_extension)

                arg_counter += 1

            if command_split[i-1] == '|':
                arg_counter = 0
        
        parsed_list_replaced.append(' '.join(command_split))

    return parsed_list_replaced


science_result = replace_arg_pipes(science2)
experienced_result = replace_arg_pipes(experienced2)
non_result = replace_arg_pipes(non2)
novice_result = replace_arg_pipes(novice2)

science_expand = parser.expand_piped_commands(science_result)
experienced_expand = parser.expand_piped_commands(experienced_result)
non_expand = parser.expand_piped_commands(non_result)
novice_expand = parser.expand_piped_commands(novice_result)

#pp.pprint(result2)

print("science:", get_accuracy(science_expand))
print("experienced:", get_accuracy(experienced_expand))
print("non:", get_accuracy(non_expand))
print("novice:", get_accuracy(novice_expand))

100%|██████████| 56/56 [00:05<00:00, 10.22it/s]


science: 65.42%
experienced: 90.95%
non: 97.49%
novice: 55.56%


In [20]:
print("science:", get_accuracy(science_expand))
print("experienced:", get_accuracy(experienced_expand))
print("non:", get_accuracy(non_expand))
print("novice:", get_accuracy(novice_expand))

science: 76.40%
experienced: 94.12%
non: 97.68%
novice: 57.78%


In [22]:
from parse import Parser

parser = Parser()

science = parser.filter_commands_with_pipe(parser.scientists_commands)
experienced = parser.filter_commands_with_pipe(parser.experienced_commands)
non = parser.filter_commands_with_pipe(parser.non_programmers_commands)
novice = parser.filter_commands_with_pipe(parser.novice_commands)

science_pipes = parser.expand_piped_commands(science)
experienced_pipes = parser.expand_piped_commands(experienced)
non_pipes = parser.expand_piped_commands(non)
novice_pipes = parser.expand_piped_commands(novice)

print(get_accuracy(science_pipes))
print(get_accuracy(experienced_pipes))
print(get_accuracy(non_pipes))
print(get_accuracy(novice_pipes))


science_pipes_expand = parser.replace_arg_expanded_pipe(science_pipes)
experienced_pipes_expand = parser.replace_arg_expanded_pipe(experienced_pipes)
non_pipes_expand = parser.replace_arg_expanded_pipe(non_pipes)
novice_pipes_expand = parser.replace_arg_expanded_pipe(novice_pipes)

print(get_accuracy(science_pipes_expand))
print(get_accuracy(experienced_pipes_expand))
print(get_accuracy(non_pipes_expand))
print(get_accuracy(novice_pipes_expand))



100%|██████████| 56/56 [00:05<00:00, 10.26it/s]


77.43%
93.67%
97.49%
84.44%


IndexError: list index out of range